- 進一步使用Gradio + colab打造地圖網頁


In [ ]:
!pip install gradio

- 資料來源:
- https://data.gov.tw/dataset/40448

In [4]:
import requests
import folium
import gradio as gr
import pandas as pd
from io import StringIO

def visualize_pm25_v4():
    """
    抓取新的 PM2.5 資料並使用 Folium 視覺化 (使用測站經緯度)，再次修正 PM2.5 欄位名稱。
    """
    url = "https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=9e565f9a-84dd-4e79-9097-d403cae1ea75&limit=100&sort=ImportDate%20desc&format=JSON"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()['records']
        df = pd.DataFrame(data)

        # 假設您的資料包含 'sitename', 'longitude', 'latitude' 和 'pm2.5' 欄位
        if 'sitename' not in df.columns or 'longitude' not in df.columns or 'latitude' not in df.columns or 'pm2.5' not in df.columns:
            return "資料中缺少必要的測站名稱、經緯度或 pm2.5 欄位。"

        # 將經緯度和 pm2.5 轉換為數值
        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
        df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce')

        # 移除經緯度或 pm2.5 為 NaN 的列
        df_cleaned = df.dropna(subset=['latitude', 'longitude', 'pm2.5'])

        if df_cleaned.empty:
            return "沒有有效的 pm2.5 資料點可以顯示。"

        # 計算地圖的中心點
        center_lat = df_cleaned['latitude'].mean()
        center_lon = df_cleaned['longitude'].mean()

        # 創建 Folium 地圖
        m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

        # 根據 pm2.5 值添加標記
        for index, row in df_cleaned.iterrows():
            station_name = row['sitename']
            latitude = row['latitude']
            longitude = row['longitude']
            pm25_value = row['pm2.5']

            color = 'green'
            if pm25_value > 35:
                color = 'orange'
            if pm25_value > 75:
                color = 'red'

            folium.CircleMarker(
                location=[latitude, longitude],
                radius=8,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7,
                tooltip=f"{station_name}: PM2.5 = {pm25_value}"
            ).add_to(m)

        # *** 重要：將 Folium 地圖物件轉換為 HTML 字串 ***
        map_html = m._repr_html_()
        return map_html
    except Exception as e:
        return f"發生未預期的錯誤：{e}"

iface_v4 = gr.Interface(
    fn=visualize_pm25_v4,
    inputs=None,
    outputs=gr.HTML(label="PM2.5 測站分佈地圖"),
    title="PM2.5 測站資料視覺化",
    description="從環境部開放資料平台抓取最新的 PM2.5 測站資料，並在地圖上顯示。",
)

iface_v4.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://82ab38bd3f014ada41.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# 把資料改成顯示pm2.5以外的項目，且至少兩項顯示在地圖上

In [ ]:
# 把新北市腳踏車地圖改用gradio呈現地圖